In [265]:
import requests
import json
from pandas.io.json import json_normalize
import numpy as np
import pandas as pd 

In [266]:
# converts converts zipcode to location coords
# dataset from https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/export/?refine.state=NY

NYC_zipcode = []
with open("zipcode.csv", 'r') as f:
        f.readline()
        for line in f:
            line = line.split(",")
            NYC_zipcode.append(str(line[0]))

NY_Zip = pd.read_csv('NYCZipGeo.csv')
print(NY_Zip.head(5))
NYC_zip_coord = []
for zipcode in NYC_zipcode:
    row = (NY_Zip.loc[NY_Zip['Zip']==int(zipcode)].values.tolist())
    lat,long = row[-1][-1].split(",")
    NYC_zip_coord.append([zipcode, float(lat), float(long)])
    



     Zip           City State   Latitude  Longitude  Timezone  \
0  14043          Depew    NY  42.904958 -78.700600        -5   
1  13827          Owego    NY  42.120274 -76.258740        -5   
2  14739     Friendship    NY  42.192384 -78.143080        -5   
3  11703  North Babylon    NY  40.733398 -73.322570        -5   
4  11707   West Babylon    NY  40.922326 -72.637078        -5   

   Daylight savings time flag               geopoint  
0                           1    42.904958, -78.7006  
1                           1   42.120274, -76.25874  
2                           1   42.192384, -78.14308  
3                           1   40.733398, -73.32257  
4                           1  40.922326, -72.637078  


In [267]:
# function to call NREL API based on year, geocoord. zipcode is only used for naming of CSV file and not API call
def get_NREL_data(year,coord,zipcode):
    url = "http://developer.nrel.gov/api/solar/nsrdb_psm3_download.csv?api_key=bXvuXx7aWzWFjrYTZwf5JOzakg9QQIUreiztEMrt"

    payload = "names="+str(year)+"&leap_day=false&interval=60&utc=false&full_name=Honored%2BUser&email=honored.user%40gmail.com&affiliation=NREL&mailing_list=true&reason=Academic&attributes=dhi%2Cdni%2Cwind_speed%2Cair_temperature&wkt=POINT"+coord
    headers = {
        'content-type': "application/x-www-form-urlencoded",
        'cache-control': "no-cache"
    }
    
    response = requests.request("POST", url, data=payload, headers=headers)
    data = (response.text) 
    
    # parsing through output by removing header files and writing it into a CSV
    data = data.split("\n")
    name = str(year) + "_" + str(zipcode) + ".csv"
    with open(name, 'w') as f:
        for line in data[2:]:
            f.write(line)
            f.write("\n")

        

In [268]:
# call NREL function for each zipcode in NYC
for zip in NYC_zip_coord:
    coord = "(" + str(zip[2]) + "+" + str(zip[1]) + ")"
    get_NREL_data(2018,coord,zip[0])

In [280]:
# aggregating zipcode data into monthly value
def calcMonthAverage(fileName):
    CSVdata = pd.read_csv(fileName)
    # convert CSV into dataframe
    monthly = {}
    for month in range(1,13):
        row = CSVdata.loc[CSVdata['Month']==month]
        monthSum = row.sum()
        monthly[month] = [str(monthSum[5]), str(monthSum[6]), str(monthSum[8])]
    
    # output result into a monthly CSV
    year = "2018"
    monthlyName = "monthly_" + fileName
    with open(monthlyName, 'w') as f:
        f.write("Year, Month, DHI, DNI, Temperature\n")
        for month in monthly:
            row = ",".join(monthly[month])
            row = year + ',' + str(month) + "," + row
            f.write(row)
            f.write("\n")

In [281]:
# calculate monthly avereage for each zipcode
def calcMonthlyAverage(zipcodes): 
    for zip in zipcodes:
        name = "2018_" + str(zip) + ".csv" 
        calcMonthAverage(name)
        
calcMonthlyAverage(NYC_zipcode)

In [283]:
# calculate annual avereage and combine them into one single pdf
def combineData(zipcodes,city):
    zipcodeDict = {}
    for zip in zipcodes:
        fileName = "monthly_2018_" + str(zip) + ".csv"
        CSVdata = pd.read_csv(fileName)
        annualSum = CSVdata.sum()
        row = [str(annualSum[2]), str(annualSum[3]), str(annualSum[4]/8760)]
        row = ",".join(row)
        zipcodeDict[zip] = row
        
    year = "2018"
    annualFile = year + "_" + city + "_Data.csv"
    with open(annualFile, 'w') as f:
        f.write("Zip, DHI, DNI, Temperature\n")
        for zip in zipcodeDict:
            row = zipcodeDict[zip]
            row = str(zip) + ',' + row
            f.write(row)
            f.write("\n")
            
combineData(NYC_zipcode,"NYC")

In [305]:
# NOTE: Returned CSV has spaces in column name for some reason. Too lazy to fix, need to manual remove in CSV file
finalData = pd.read_csv("2018_NYC_Data.csv")
finalData.head()
result = finalData[ (finalData['DHI']>= 538737) & (finalData['DNI'] >= 1428080) ]
print(result)

       Zip     DHI      DNI  Temperature    Total
1    11213  540131  1447343    11.791541  1987474
72   11101  541582  1436014    11.814201  1977596
87   11222  541582  1436014    11.814201  1977596
97   11206  540131  1447343    11.791541  1987474
168  11221  540131  1447343    11.791541  1987474
172  11216  540131  1447343    11.791541  1987474
188  11211  541582  1436014    11.814201  1977596
